PyQubit Ai 

Developer:Mohammad Mahdi Omidvar 

In [ ]:
!pip install flask flask-cors ollama db-sqlite3
#ollama pull gemma3:4b

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from ollama import chat
import os
import sqlite3


app = Flask(__name__)
CORS(app)

DATABASE = 'chatbot.db'

def init_db():
    if not os.path.exists(DATABASE):
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS PyQubit (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                model TEXT,
                user_id TEXT,
                req TEXT,
                res TEXT
            )
        ''')
        conn.commit()
        conn.close()

init_db()

@app.route('/', methods=['GET'])
def home():
    chat1= request.args.get('chat1')

    mobile = request.args.get('mobile')

    if not chat1 or not mobile:
        return jsonify({"error"}), 400

    contex='''     
your name is PyQubit AI and You are AI 
    '''

    messages = [{'role': 'system', 'content': contex}]
    
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    try:
        cursor.execute("""
            SELECT req, res FROM PyQubit
            WHERE user_id = ?
            ORDER BY id DESC
            LIMIT 40
        """, (mobile,))
        results = cursor.fetchall()
        for row in reversed(results):
            messages.append({"role": "user", "content": row[0]})
            messages.append({"role": "assistant", "content": row[1]})
    except Exception as e:
        print(f"history error {e}")

    soal = chat1
    messages.append({'role': 'user', 'content': soal})

    try:
        stream = chat(
            model='gemma3:4b',
            messages=messages
        )

        out = stream.message.content 

        cursor.execute("""
            INSERT INTO PyQubit
            (model, user_id, req, res)
            VALUES (?, ?, ?, ?)
        """, ('gemma3:4b', mobile, chat1, out))
        conn.commit()

    except Exception as e:
        print(f"save error {e}")
        out = "❌error "

    finally:
        conn.close()

    out = out.replace("\n", "<br>").replace("\r", "<br>").replace("\\", "'").replace('"', "'")
    out = out.replace("*", "◾").replace("####", "✅").replace("###", "✅").replace("##", "✅")

    return out

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.21.214.186:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2025 16:52:04] "GET /?chat1=hello&mobile=09876545678987 HTTP/1.1" 200 -
